**RNN을 이용한 네이버 영화 리뷰 감성 분석**
//2~7은 NLP에서 필요한 과정
1. 데이터 로드
2. 중복 데이터 제거
3. 정규식 이용한 한글 및 공백 이외 문자 제거
4. Null 데이터 제거
5. 형태소 분석기를 활용한 토큰화 및 불용어 제거 -> 한국어 NLP 특화된 처리가 필요한 과정
6. Vocab 생성 및 단어 인덱스 형태의 데이터셋 생성
7. 데이터에 대한 padding 진행
8. PyTorch 데이터셋 생성
9. 모델 정의, 학습 및 테스트 함수 정의;
10. 학습 및 테스트 진행


**자연언어 종류 3가지**
**고립어: 한자를 사용하는 중국어와 같이 단어의 형태가 시제, 인칭 변화 등에 따라 바뀌지 않고 고정된 형태의 언어**
**굴절어: 영어, 독일어 등과 같이 시제, 인칭 벼화 등에 따라 단어의 형태가 변하는 언어 (띄어쓰기로 split해도 높은 퀄리티의 토큰 생성)**
**교착어: 한국어, 일본어 등과 같이 단어에 어미가 붙어 시제, 인칭 등의 뜻이 최종 결정되는 언어**

## 한국어 텍스트 토큰화의 어려움
**한국어 텍스트는 split을 수행하여 토큰화를 하였을 때, 조사가 붙어있어 서로 다른 단어로 인식**
**한국어는 띄어쓰기 맞춤법이 있지만 띄어쓰기가 잘 지켜지지 않는 corpus도 많으므로 기본적인 split으로 만족스러운 토큰들을 얻기 어려움**
### -> 한국어 텍스트에서 영어와 유사한 토큰들을 얻기 위해서는 형태소 단위의 분리기 필요함

**형태소: 의미를 가진 가장 작은 단위의 말 (자립 형태소, 의존 형태소)**
**형태소 분석기: 원문 텍스트에서 형태소를 추출하여 토큰화 해주는 모듈, 프로그램**
**KoNLPy, 카카오의 khaii 등이 있음**

**konlpy.tag에 대양한 형태소 분석기가 있다. 작동 방식과 세부 성능에 차이가 있다**
**자주 쓰이는 분석기들**
**Okt: Open Korean Text의 줄임말로써 예전에는 Twitter 분석기라고 불렸음. "ㅋㅋ", "ㄴㄴ"등과 같은 인터넷 용어들도 비교적 분석이 가능한 장점이 있음**
**Kkma: 꼬꼬마 형탳소 분석기의 줄임말. 어미 분석을 자세하게 수행해준다는 것이 장점**
**Mecab: 일본어 형태소 분석기 Mecab을 한국어 버전으로 구현한 것(+가장 무난하다)**

In [ ]:
!pip install konlpy

**자연언어는 세 종류로 구분됨**


In [ ]:
from konlpy.tag import Okt

# KoNLPy 형태소 분석기들은 moprhs 메서드에 문장을 입력하여 형태소 분석이 가능
# pos 메서드를 통해 형태소 분석 및 품사 태깅이 가능
# 품사 태깅: 각 형태소마다 품사를 예측한 후 형태소-품사 pair을 만드는 것
# 특정 품사는 제거하고 특정 품사는 남기는 등의 전처리 가능
okt = Okt()
print('Okt 형태소 분석 :', okt.morphs)("한국어 단어는 형태소들로 구성되어 있다.")
print('Okt 품사 태깅 :', okt.pos("한국어 단어는 형태소들로 구성되어 있다."))


In [ ]:
from konlpy.tag import Kkma
kkma = Kkma()
print("Kkma 형태소 분석 :", kkma.morphs("힌극어 단어는 형태소들로 구성되어 있다."))
print('Kkma 품사 태깅 :', kkma.pos("한국어 단어는 형태소들로 구성되어 있다."))

#구현 필요

**네이버 영화 리뷰 감성분석**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from collections import Counter
#device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

ModuleNotFoundError: No module named 'konlpy'

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [ ]:
# 데이터셋 로드
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

In [ ]:
# 학습 데이터 확인
print ('Train data len :',len(train_data))
train_data[:5]

In [ ]:
# 테스트 데이터 확인
print ('Test data len :',len(test_data))
test_data[:5]

In [ ]:
# 구현 필요
# 학습 데이터에서 중복되는 데이터 제거 및 이후 상태 확인 
train_data.drop_duplicates(subsets=['document'], inplace=True) 

In [ ]:
train_data.groupby('label').size()

In [ ]:
#구현 필요
# 한글 및 공백만 남도록 전처리 및 결과 확인
train_data['document'] = train_data['document'].str.replace("[ㄱ-ㅎㅏ-ㅣ가-힣 ","") # 한글 및 공백 이외 제거
train_data[:5]

In [ ]:
#구현 필요
# 정규식으로 전처리 후 아예 공백만 남은 문장이 있을 수 있다
# -> Null 으로 변경 후 일괄 제거
# white space 데이터를 empty value로 변경
train_data['document'] = train_data['document'].str.replace('^ +', "")
train_data['document'].replace('', np.nan, inplace=True)
print(train_data.isnull().sum())

In [ ]:
train_data.loc[train_data.document.isnull()][:5]

In [ ]:
#구현 필요
# null값 제거
train_data = train_data.dropna(how= 'any')
print(len(train_data))

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

In [ ]:
# 불용어 정의 및 형태소 분석기를 이용한 토큰화 진행
# stem = True 옵션으로 형태소 분석을 수행할 시 형태소 원형이 반환된다.
# 되나요 -> 되다 와 같은 형식으로 반환되는 형태소가 바뀜
okt = Okt()
X_train = []
for sentence in tqdm(train_data['document']):
    tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_train.append(stopwords_removed_sentence)
print(X_train[:3])

In [ ]:
# 데스트 데이터에 대해서도 토큰화 진행
# 기본적으로 각종 전처리 및 vocab 생성은 학습 데이터에 대해서만 이루어짐
# 데스트 데이터는 실제 모델 서비스 시 무엇이 들어올지 모르지 모르는 상태로 가정
# 테스트 데이터는 숫자형 데이터도 존재하는 등 데이터 정제가 되지 않은 상태이므로 string으로 캐스팅 후 토큰화 수행
X_test = []
for sentence in tqdm(test_data['document']):
    tokenized_sentence = okt.morphs(str(sentence), stem=True) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    X_test.append(stopwords_removed_sentence)

In [ ]:
# 토큰화 된 단어들에 대해서 vocab 생성 및 단어 인덱스 형태의 데이터셋 생성
from collections import Counter
def tokenize(x_train,y_train,x_val,y_val):
    word_list = []

    for sent in x_train:
      for word in sent:        
          word_list.append(word)
  
    corpus = Counter(word_list)
    # sorting on the basis of most common words
    corpus_ = sorted(corpus,key=corpus.get,reverse=True)[:10000]
    # creating a dict
    onehot_dict = {w:i+1 for i,w in enumerate(corpus_)}
    
    # tokenize
    final_list_train,final_list_test = [],[]
    for sent in x_train:
            final_list_train.append([onehot_dict[word] for word in sent
                                     if word in onehot_dict.keys()])
    for sent in x_val:
            final_list_test.append([onehot_dict[word] for word in sent 
                                    if word in onehot_dict.keys()])
   
    return np.array(final_list_train), np.array(y_train),np.array(final_list_test), np.array(y_val),onehot_dict

In [ ]:
#구현 필요 
# 데이터셋 생성 및 문장 길이(단어 단위)에 대한 통계 분석
x_train, y_train, X_test, y_test, vocab = tokenize(X_train, train_data['label'], X_test, test_data['label'])


In [ ]:
# 데이터셋 생성 및 문장 길이(단어 단위)에 대한 통계 분석
rev_len = [len(i) for i in x_train]
pd.Series(rev_len).hist()
plt.show()
pd.Series(rev_len).describe()

In [ ]:
# 대부분의 문장을 그대로 담을 수 있는 길이 50으로 데이터 padding 진행
def padding_(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

# 50 확인
x_train_pad = padding_(x_train,50)
x_test_pad = padding_(x_test,50)

In [ ]:
# 데이터셋 생성 (IMDb 실습 코드와 동일)
# create Tensor datasets
train_data = TensorDataset(torch.from_numpy(x_train_pad), torch.from_numpy(y_train))
test_data = TensorDataset(torch.from_numpy(x_test_pad), torch.from_numpy(y_test))

# dataloaders
batch_size = 50

# make sure to SHUFFLE your data
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)

In [ ]:
# 모델 정의
class GRU_model(nn.Module):
    def __init__(self, n_layers, hidden_dim, n_vocab, embed_dim, n_classes, device):
        super(GRU_model, self).__init__()
        self.n_layers = n_layers
        self.hidden_dim = hidden_dim
        self.device = device

        self.embed = nn.Embedding(n_vocab, embed_dim)
        self.gru = nn.GRU(embed_dim, self.hidden_dim,
                          num_layers=self.n_layers,
                          batch_first=True)
        self.out = nn.Linear(self.hidden_dim, n_classes)

    def forward(self, x):
        x = self.embed(x)
        h_0 = self._init_state(batch_size=x.size(0)) # 첫번째 히든 스테이트를 0벡터로 초기화
        x, _ = self.gru(x, h_0)  # GRU의 리턴값은 (배치 크기, 시퀀스 길이, 은닉 상태의 크기)
        h_t = x[:,-1,:] # (배치 크기, 은닉 상태의 크기)의 텐서로 크기가 변경됨. 즉, 마지막 time-step의 은닉 상태만 가져온다.
        logit = self.out(h_t)  # (배치 크기, 은닉 상태의 크기) -> (배치 크기, 출력층의 크기)
        return logit

    def _init_state(self, batch_size):
        new_state = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(self.device)
        return new_state

In [3]:
# Device 설정 (IMDb 실습 코드와 동일)
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

NameError: name 'torch' is not defined

In [ ]:
# 하이퍼파라미터 설정 및 모델 객체 생성
n_layers = 1
vocab_size = len(vocab) + 1  # extra 1 for <pad>
hidden_dim = 128
embed_dim = 100
n_classes = 2

model = GRU_model(n_layers, hidden_dim, vocab_size, embed_dim, n_classes, device).to(device)

In [ ]:
# 학습 및 테스트 함수 정의
def train(model, criterion, optimizer, data_loader):
    model.train()
    train_loss = 0
    for i, (x, y) in enumerate(data_loader):
        x, y = x.to(device), y.to(device)
        
        optimizer.zero_grad()
        logit = model(x)
        loss = criterion(logit, y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * x.size(0)
      
    return train_loss / len(data_loader.dataset)

def evaluate(model, data_loader):
    model.eval()
    corrects, total_loss = 0, 0
    for i, (x, y) in enumerate(data_loader):
        x, y = x.to(device), y.to(device)

        logit = model(x)
        corrects += (logit.max(1)[1].view(y.size()).data == y.data).sum()
    size = len(data_loader.dataset)
    
    avg_accuracy = 100.0 * corrects / size
    return avg_accuracy

In [ ]:
# 학습 및 테스트 loop
num_epochs = 10
lr = 0.001

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for e in range(1, num_epochs+1):
    train_loss = train(model, criterion, optimizer, train_loader)
    test_accuracy = evaluate(model, test_loader)

    print("[Epoch: %d] train loss : %5.2f | test accuracy : %5.2f" % (e, train_loss, test_accuracy))

**BERT 기반 감성분석**

**BERT를 활용하기 위해 Transformers 설치 및 데이터셋 로드**

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import re
import urllib.request
import numpy as np

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

In [3]:
train_data = pd.read_table('ratings_train.txt')
test_data = pd.read_table('ratings_test.txt')

**테스트 데이터를 전처리 해주지 않으면 토큰화 과정에서 에러가 발생함을 확인하여 학습 데이터와 같은 형식으로 일괄 전처리 진행**
**실제 서비스 상황을 가정한다면 테스트 데이터는 매번 입력하기 전에 전처리하여 만약 빈 입력이 되면 스킵하는 것으로 처리**

In [ ]:
train_data.drop_duplicates(subset=['document'], inplace=True)  # 중복 데이터 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")  # 한글 및 공백 이외 제거
train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
train_data['document'].replace('', np.nan, inplace=True)
train_data = train_data.dropna(how = 'any')  # null값 제거

In [ ]:
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")  # 한글 및 공백 이외 제거
test_data['document'] = test_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
test_data['document'].replace('', np.nan, inplace=True)
test_data = test_data.dropna(how = 'any')  # null값 제거

In [ ]:
# 한국어 대상 pre-trained BERT 모델이 여러 가지 있으나 klue/bert-base 활용하여 실습 진행
# 학습 속도 및 메모리 문제 때문에 데이터셋은 10000개 까지만 샘플링하여 활용


from transformers import AutoModelForSequenceClassification, AutoTokenizer
#구현 필요
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
# BERT는 word piece라는 sub-word 토큰화 알고리즘을 사용하기 때문에 별도의 형태소 분석 없이 바로 토큰화 가능

sampled_train_data = train_data[:10000]
sampled_test_data = test_data[:10000]

#구현 필요
train_tokens = tokenizer(list(sampled_train_data['document']), padding='max_length', truncation=True, return_tensors="pt", add_special_tokens=True)
test_tokens = tokenizer(list(sampled_test_data['document']), padding='max_length', truncation=True, return_tensors="pt", add_special_tokens=True)
print(tokenizer.convert_ids_to_tokens(train_tokens['input_ids'][0]))

In [ ]:
#Trainer에 입력해주기 위한 데이터셋 생성
import torch

class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

#구현 필요
train_dataset = BERTDataset(train_tokens, list(sampled_train_data['label']))
test_dataset = BERTDataset(test_tokens, list(sampled_test_data['label']))

In [ ]:
# 학습 환경 설정 및 정확도 확인을 위한 별도 함수 작성
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=10,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=250,
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)    
    return {
        'accuracy': acc
    }
# Trainer 객체 생성 및 학습 진행
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset,           # evaluation dataset
    compute_metrics=compute_metrics      # additional evaluation metrics
)

trainer.train()

In [ ]:
trainer.evaluate()